In [0]:
letters=sc.parallelize(['a' , 'b', 'c', 'd', 'a', 'f', 'd', 'c', 'b', 'c', 'c'], 3) 
#letters
letters.first()
#print("1st element: "+str(letters.first()))

Out[1]: 'a'

In [0]:
letters.collect()
#letters

Out[3]: ['a', 'b', 'c', 'd', 'a', 'f', 'd', 'c', 'b', 'c', 'c']

In [0]:
letters.filter(lambda x: x=='a').collect()
#filteredLetters=letters.filter(lambda x: x=='a')
#filteredLetters

Out[4]: ['a', 'a']

In [0]:
pairs=letters.map(lambda x: (x,1))
#print(pairs.collect())
#print(pairs.reduceByKey(lambda x,y: x+y).collect())

pairs.reduceByKey(lambda x,y: x+y).collect()

# if you pass non-commutative and/or non-associative lambdas, the results are unpredictable
#pairs.reduceByKey(lambda x,y: 2*x+y).collect()
#pairs.reduceByKey(lambda x,y: x+2*y).collect()

# This won't work. Try it!
# letters.reduceByKey(lambda x,y: x+y).collect()
# The following won't work either
# pairs.reduceByKey(lambda x: x+1).collect()


Out[14]: [('d', 3), ('b', 3), ('a', 3), ('c', 11), ('f', 1)]

In [0]:
# For more info on dbfs commands:   https://docs.databricks.com/_static/notebooks/dbutils.html
dbutils.fs.mkdirs('/FileStore/DemoData')               # folder is only created if it doesn't already exist
dbutils.fs.rm('/FileStore/DemoData/PeoplePoem.txt')    # in case file already exists
dbutils.fs.put('/FileStore/DemoData/PeoplePoem.txt',
           'People are not as beautiful as they look, as they walk or as they talk.\n They are only as beautiful as they love, as they care as they share.')

Wrote 141 bytes.
Out[2]: True

In [0]:
lines=sc.textFile('/FileStore/DemoData/PeoplePoem.txt')
lines.map(lambda line: line.split(" ")).collect()       # just to show the difference between map and flatMap

Out[3]: [['People',
  'are',
  'not',
  'as',
  'beautiful',
  'as',
  'they',
  'look,',
  'as',
  'they',
  'walk',
  'or',
  'as',
  'they',
  'talk.'],
 ['',
  'They',
  'are',
  'only',
  'as',
  'beautiful',
  'as',
  'they',
  'love,',
  'as',
  'they',
  'care',
  'as',
  'they',
  'share.']]

In [0]:
lines.flatMap(lambda line: line.split(" ")).collect() 

Out[4]: ['People',
 'are',
 'not',
 'as',
 'beautiful',
 'as',
 'they',
 'look,',
 'as',
 'they',
 'walk',
 'or',
 'as',
 'they',
 'talk.',
 '',
 'They',
 'are',
 'only',
 'as',
 'beautiful',
 'as',
 'they',
 'love,',
 'as',
 'they',
 'care',
 'as',
 'they',
 'share.']

In [0]:
counts=lines.flatMap(lambda line: line.split(" "))\
            .map(lambda word: (word, 1))\
            .reduceByKey(lambda x,y: x+y)
counts.collect()

Out[5]: [('are', 2),
 ('as', 8),
 ('walk', 1),
 ('', 1),
 ('They', 1),
 ('only', 1),
 ('share.', 1),
 ('People', 1),
 ('not', 1),
 ('beautiful', 2),
 ('they', 6),
 ('look,', 1),
 ('or', 1),
 ('talk.', 1),
 ('love,', 1),
 ('care', 1)]

In [0]:
# decode is used to convert bytes object to string
print(counts.toDebugString().decode())

(2) PythonRDD[10] at collect at <command-2288209583515534>:4 []
 |  MapPartitionsRDD[9] at mapPartitions at PythonRDD.scala:183 []
 |  ShuffledRDD[8] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[7] at reduceByKey at <command-2288209583515534>:1 []
    |  PythonRDD[6] at reduceByKey at <command-2288209583515534>:1 []
    |  /FileStore/DemoData/PeoplePoem.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0 []
    |  /FileStore/DemoData/PeoplePoem.txt HadoopRDD[2] at textFile at NativeMethodAccessorImpl.java:0 []


In [0]:
counts.cache()

Out[7]: PythonRDD[10] at collect at <command-2288209583515534>:4

In [0]:
dbutils.fs.rm('/FileStore/DemoData/output',recurse=True)
counts.saveAsTextFile('/FileStore/DemoData/output')
dbutils.fs.ls('/FileStore/DemoData/output')

Out[9]: [FileInfo(path='dbfs:/FileStore/DemoData/output/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1655908767000),
 FileInfo(path='dbfs:/FileStore/DemoData/output/part-00000', name='part-00000', size=79, modificationTime=1655908767000),
 FileInfo(path='dbfs:/FileStore/DemoData/output/part-00001', name='part-00001', size=115, modificationTime=1655908767000)]

In [0]:
# Print all lines in all files with size greater than 0 (to exclude _SUCCESS)
for f in dbutils.fs.ls('/FileStore/DemoData/output'):
  if f[2]>0:
#    print(sc.textFile(f[0]).collect())
    for line in sc.textFile(f[0]).collect():
      print(line)

["('are', 2)", "('as', 8)", "('walk', 1)", "('', 1)", "('They', 1)", "('only', 1)", "('share.', 1)"]
["('People', 1)", "('not', 1)", "('beautiful', 2)", "('they', 6)", "('look,', 1)", "('or', 1)", "('talk.', 1)", "('love,', 1)", "('care', 1)"]


In [0]:
dbutils.fs.rm('/FileStore/DemoData/output',recurse=True)

Out[12]: True

In [0]:
# Let's validate that the raw data is as we expect it first
rawData=sc.textFile("/FileStore/DemoData/cars.txt")
rawData.collect()

Out[13]: ['make\tModel\tMPG\tCylinders\tEngine Disp\tHorsepower\tWeight\tAccelerate\tYear\tOrigin',
 'bmw\tbmw 2002\t26\t4\t121\t113\t2234\t12.5\t70\tEuropean ',
 'peugeot\tpeugeot 504\t25\t4\t110\t87\t2672\t17.5\t70\tEuropean ',
 'saab\tsaab 99e\t25\t4\t104\t95\t2375\t17.5\t70\tEuropean ',
 'datsun\tdatsun pl510\t27\t4\t97\t88\t2130\t14.5\t70\tJapanese ',
 'toyota\ttoyota corona\t25\t4\t113\t95\t2228\t14\t71\tJapanese',
 'amc\tamc gremlin\t19\t6\t232\t100\t2634\t13\t71\tAmerican ',
 'amc\tamc hornet\t18\t6\t258\t110\t2962\t13.5\t71\tAmerican ']

In [0]:
carsData=rawData.map(lambda line: line.split("\t"))\
                .filter(lambda row: row[0]!='make')\
                .map(lambda row: row[:-1] + [ row[-1].strip() ] )

for car in carsData.collect():
  print(car)

['bmw', 'bmw 2002', '26', '4', '121', '113', '2234', '12.5', '70', 'European']
['peugeot', 'peugeot 504', '25', '4', '110', '87', '2672', '17.5', '70', 'European']
['saab', 'saab 99e', '25', '4', '104', '95', '2375', '17.5', '70', 'European']
['datsun', 'datsun pl510', '27', '4', '97', '88', '2130', '14.5', '70', 'Japanese']
['toyota', 'toyota corona', '25', '4', '113', '95', '2228', '14', '71', 'Japanese']
['amc', 'amc gremlin', '19', '6', '232', '100', '2634', '13', '71', 'American']
['amc', 'amc hornet', '18', '6', '258', '110', '2962', '13.5', '71', 'American']


In [0]:
# This has the same effect as the command above, but applies strip to all the fields of each row, not just to the origin
carsData=rawData.map(lambda line: line.split("\t"))\
                .filter(lambda line: line[0]!='make')\
                .map(lambda row: [ e.strip() for e in row])

for car in carsData.collect():
  print(car)

['bmw', 'bmw 2002', '26', '4', '121', '113', '2234', '12.5', '70', 'European']
['peugeot', 'peugeot 504', '25', '4', '110', '87', '2672', '17.5', '70', 'European']
['saab', 'saab 99e', '25', '4', '104', '95', '2375', '17.5', '70', 'European']
['datsun', 'datsun pl510', '27', '4', '97', '88', '2130', '14.5', '70', 'Japanese']
['toyota', 'toyota corona', '25', '4', '113', '95', '2228', '14', '71', 'Japanese']
['amc', 'amc gremlin', '19', '6', '232', '100', '2634', '13', '71', 'American']
['amc', 'amc hornet', '18', '6', '258', '110', '2962', '13.5', '71', 'American']


In [0]:
countByOrigin=carsData.map(lambda x: (x[9],1))\
                      .reduceByKey(lambda x,y: x+y)
countByOrigin.collect()

Out[16]: [('European', 3), ('American', 2), ('Japanese', 2)]

In [0]:
americanCars=carsData.filter(lambda x: x[9]=='American')
americanCars.collect()

Out[17]: [['amc',
  'amc gremlin',
  '19',
  '6',
  '232',
  '100',
  '2634',
  '13',
  '71',
  'American'],
 ['amc',
  'amc hornet',
  '18',
  '6',
  '258',
  '110',
  '2962',
  '13.5',
  '71',
  'American']]

In [0]:
originWeightSum=carsData.map(lambda x: (x[9],int(x[6])))\
                        .combineByKey(lambda x: (x, 1),\
                                      lambda a,x: (a[0] + x, a[1] + 1),\
                                      lambda a,x: (a[0]+x[0], a[1]+x[1]))
#originWeightSum.collectAsMap()
originWeightSum.collect()

Out[18]: [('European', (7281, 3)), ('American', (5596, 2)), ('Japanese', (4358, 2))]

In [0]:
originWeightAvg=originWeightSum.map(lambda x: (x[0], x[1][0]/x[1][1])) 
originWeightAvg.collectAsMap()

Out[19]: {'European': 2427.0, 'American': 2798.0, 'Japanese': 2179.0}

In [0]:
apple = sc.textFile('dbfs:/FileStore/tables/appl_daily.csv')
firstRow = apple.first()
apple = apple.filter(lambda row:row != firstRow)
apple = apple.map(lambda row: row.split(","))

face = sc.textFile('dbfs:/FileStore/tables/face_daily.csv')
firstRow = face.first()
face = face.filter(lambda row:row != firstRow)
face = face.map(lambda row: row.split(","))

In [0]:
apple.take(10)

Out[2]: [['2015-07-09',
  '123.849998',
  '124.059998',
  '119.220001',
  '120.07',
  '77821600',
  '120.07'],
 ['2015-07-08',
  '124.480003',
  '124.639999',
  '122.540001',
  '122.57',
  '60490200',
  '122.57'],
 ['2015-07-07',
  '125.889999',
  '126.150002',
  '123.769997',
  '125.690002',
  '46716100',
  '125.690002'],
 ['2015-07-06',
  '124.940002',
  '126.230003',
  '124.849998',
  '126.00',
  '27900200',
  '126.00'],
 ['2015-07-02',
  '126.43',
  '126.690002',
  '125.769997',
  '126.440002',
  '27122500',
  '126.440002'],
 ['2015-07-01',
  '126.900002',
  '126.940002',
  '125.989998',
  '126.599998',
  '30128600',
  '126.599998'],
 ['2015-06-30',
  '125.57',
  '126.120003',
  '124.860001',
  '125.43',
  '43849800',
  '125.43'],
 ['2015-06-29',
  '125.459999',
  '126.470001',
  '124.480003',
  '124.529999',
  '48911400',
  '124.529999'],
 ['2015-06-26',
  '127.669998',
  '127.989998',
  '126.510002',
  '126.75',
  '42111000',
  '126.75'],
 ['2015-06-25',
  '128.860001',
  '129.19

In [0]:
face.take(10)

Out[3]: [['2015-12-11',
  '104.150002',
  '104.339996',
  '101.910004',
  '102.120003',
  '26286300',
  '102.120003'],
 ['2015-12-10',
  '105.050003',
  '106.400002',
  '104.410004',
  '105.419998',
  '17414500',
  '105.419998'],
 ['2015-12-09',
  '106.18',
  '106.440002',
  '103.550003',
  '104.599998',
  '23123000',
  '104.599998'],
 ['2015-12-08',
  '104.00',
  '106.910004',
  '103.949997',
  '106.489998',
  '20365300',
  '106.489998'],
 ['2015-12-07',
  '106.480003',
  '106.830002',
  '104.660004',
  '105.610001',
  '15403400',
  '105.610001'],
 ['2015-12-04',
  '104.809998',
  '107.730003',
  '104.110001',
  '106.18',
  '20941200',
  '106.18'],
 ['2015-12-03',
  '106.269997',
  '106.849998',
  '103.349998',
  '104.379997',
  '23171000',
  '104.379997'],
 ['2015-12-02',
  '107.00',
  '107.919998',
  '105.790001',
  '106.07',
  '24295200',
  '106.07'],
 ['2015-12-01',
  '104.830002',
  '107.150002',
  '104.50',
  '107.120003',
  '22715600',
  '107.120003'],
 ['2015-11-30',
  '105.83

In [0]:
# Join - first attempt
apple.join(face).take(10)

Out[10]: [('2015-07-09', ('123.849998', '86.730003')),
 ('2015-06-23', ('127.480003', '84.959999')),
 ('2015-06-18', ('127.230003', '81.639999')),
 ('2015-06-09', ('126.699997', '80.599998')),
 ('2015-06-08', ('128.899994', '81.82')),
 ('2015-06-03', ('130.660004', '81.349998')),
 ('2015-06-01', ('130.279999', '79.300003')),
 ('2015-05-22', ('131.600006', '80.349998')),
 ('2015-05-21', ('130.070007', '80.190002')),
 ('2015-05-18', ('128.380005', '80.330002'))]

In [0]:
# That doesn't look like what we wanted...

In [0]:
# A better look at the results suggests that we need to prepare the RDDs involved in the joins as RDDs of pairs

# Here's how to do that. First we prepare the apple RDD:
apple_date_key = apple.map(lambda x: (x[0], x[1:]))
apple_date_key.take(10)

Out[6]: [('2015-07-09',
  ['123.849998', '124.059998', '119.220001', '120.07', '77821600', '120.07']),
 ('2015-07-08',
  ['124.480003', '124.639999', '122.540001', '122.57', '60490200', '122.57']),
 ('2015-07-07',
  ['125.889999',
   '126.150002',
   '123.769997',
   '125.690002',
   '46716100',
   '125.690002']),
 ('2015-07-06',
  ['124.940002', '126.230003', '124.849998', '126.00', '27900200', '126.00']),
 ('2015-07-02',
  ['126.43',
   '126.690002',
   '125.769997',
   '126.440002',
   '27122500',
   '126.440002']),
 ('2015-07-01',
  ['126.900002',
   '126.940002',
   '125.989998',
   '126.599998',
   '30128600',
   '126.599998']),
 ('2015-06-30',
  ['125.57', '126.120003', '124.860001', '125.43', '43849800', '125.43']),
 ('2015-06-29',
  ['125.459999',
   '126.470001',
   '124.480003',
   '124.529999',
   '48911400',
   '124.529999']),
 ('2015-06-26',
  ['127.669998', '127.989998', '126.510002', '126.75', '42111000', '126.75']),
 ('2015-06-25',
  ['128.860001', '129.199997', '127.5

In [0]:
# Next we prepare the facebook RDD:
face_date_key = face.map(lambda x: (x[0], x[1:]))
face_date_key.take(10)

Out[7]: [('2015-12-11',
  ['104.150002',
   '104.339996',
   '101.910004',
   '102.120003',
   '26286300',
   '102.120003']),
 ('2015-12-10',
  ['105.050003',
   '106.400002',
   '104.410004',
   '105.419998',
   '17414500',
   '105.419998']),
 ('2015-12-09',
  ['106.18',
   '106.440002',
   '103.550003',
   '104.599998',
   '23123000',
   '104.599998']),
 ('2015-12-08',
  ['104.00',
   '106.910004',
   '103.949997',
   '106.489998',
   '20365300',
   '106.489998']),
 ('2015-12-07',
  ['106.480003',
   '106.830002',
   '104.660004',
   '105.610001',
   '15403400',
   '105.610001']),
 ('2015-12-04',
  ['104.809998', '107.730003', '104.110001', '106.18', '20941200', '106.18']),
 ('2015-12-03',
  ['106.269997',
   '106.849998',
   '103.349998',
   '104.379997',
   '23171000',
   '104.379997']),
 ('2015-12-02',
  ['107.00', '107.919998', '105.790001', '106.07', '24295200', '106.07']),
 ('2015-12-01',
  ['104.830002',
   '107.150002',
   '104.50',
   '107.120003',
   '22715600',
   '107.120

In [0]:
# Finally, we join:
apple_date_key.join(face_date_key).take(10)

Out[8]: [('2015-07-09',
  (['123.849998', '124.059998', '119.220001', '120.07', '77821600', '120.07'],
   ['86.730003',
    '87.599998',
    '85.650002',
    '85.879997',
    '23307400',
    '85.879997'])),
 ('2015-06-23',
  (['127.480003',
    '127.610001',
    '126.879997',
    '127.029999',
    '30137100',
    '127.029999'],
   ['84.959999',
    '87.970001',
    '84.760002',
    '87.879997',
    '50667900',
    '87.879997'])),
 ('2015-06-18',
  (['127.230003',
    '128.309998',
    '127.220001',
    '127.879997',
    '35241100',
    '127.879997'],
   ['81.639999', '83.190002', '81.57', '82.910004', '26782600', '82.910004'])),
 ('2015-06-09',
  (['126.699997',
    '128.080002',
    '125.620003',
    '127.419998',
    '55771500',
    '127.419998'],
   ['80.599998', '81.190002', '79.32', '80.669998', '16497500', '80.669998'])),
 ('2015-06-08',
  (['128.899994',
    '129.210007',
    '126.830002',
    '127.800003',
    '52538200',
    '127.800003'],
   ['81.82', '82.00', '80.129997', '8

In [0]:
# Now, the RDD contains all the data we need. Great!

# However, the final structure contains way too much nestedness! 
# Imagine how complex it would become if we were to join the result with another RDD (and then another, and another, ...)

# As part of the workbook, you will be asked to join RDDs. Don't forget to remove some of the nestedness from the result.

In [0]:
------  END OF DEMO   ---------------

In [0]:
apple.map(lambda row: ('*',(float(row[2]),float(row[3])))).reduceByKey(lambda x,y: (x[0] if x[0]>y[0] else y[0], x[1] if x[1]<y[1] else y[1])).map(lambda x: x[1]).collect()

Out[5]: [(705.070023, 11.00008)]

In [0]:
apple.map(lambda x : (x[0], x[2])).collect()

Out[2]: [('2015-07-09', '124.059998'),
 ('2015-07-08', '124.639999'),
 ('2015-07-07', '126.150002'),
 ('2015-07-06', '126.230003'),
 ('2015-07-02', '126.690002'),
 ('2015-07-01', '126.940002'),
 ('2015-06-30', '126.120003'),
 ('2015-06-29', '126.470001'),
 ('2015-06-26', '127.989998'),
 ('2015-06-25', '129.199997'),
 ('2015-06-24', '129.800003'),
 ('2015-06-23', '127.610001'),
 ('2015-06-22', '128.059998'),
 ('2015-06-19', '127.82'),
 ('2015-06-18', '128.309998'),
 ('2015-06-17', '127.879997'),
 ('2015-06-16', '127.849998'),
 ('2015-06-15', '127.239998'),
 ('2015-06-12', '128.330002'),
 ('2015-06-11', '130.179993'),
 ('2015-06-10', '129.339996'),
 ('2015-06-09', '128.080002'),
 ('2015-06-08', '129.210007'),
 ('2015-06-05', '129.690002'),
 ('2015-06-04', '130.580002'),
 ('2015-06-03', '130.940002'),
 ('2015-06-02', '130.660004'),
 ('2015-06-01', '131.389999'),
 ('2015-05-29', '131.449997'),
 ('2015-05-28', '131.949997'),
 ('2015-05-27', '132.259995'),
 ('2015-05-26', '132.910004'),
 ('2

In [0]:
apple.map(lambda x: ('*',x)).reduceByKey(lambda x,y: x if int(x[5])>int(y[5]) else y ).map(lambda x: (x[1][0], x[1][5])).collect()

Out[5]: [('2000-09-29', '1855410200')]

In [0]:
apple.map(lambda x : (x[0], x[1], x[4], x[6])).collect()

Out[7]: [('2015-07-09', '123.849998', '120.07', '120.07'),
 ('2015-07-08', '124.480003', '122.57', '122.57'),
 ('2015-07-07', '125.889999', '125.690002', '125.690002'),
 ('2015-07-06', '124.940002', '126.00', '126.00'),
 ('2015-07-02', '126.43', '126.440002', '126.440002'),
 ('2015-07-01', '126.900002', '126.599998', '126.599998'),
 ('2015-06-30', '125.57', '125.43', '125.43'),
 ('2015-06-29', '125.459999', '124.529999', '124.529999'),
 ('2015-06-26', '127.669998', '126.75', '126.75'),
 ('2015-06-25', '128.860001', '127.50', '127.50'),
 ('2015-06-24', '127.209999', '128.110001', '128.110001'),
 ('2015-06-23', '127.480003', '127.029999', '127.029999'),
 ('2015-06-22', '127.489998', '127.610001', '127.610001'),
 ('2015-06-19', '127.709999', '126.599998', '126.599998'),
 ('2015-06-18', '127.230003', '127.879997', '127.879997'),
 ('2015-06-17', '127.720001', '127.300003', '127.300003'),
 ('2015-06-16', '127.029999', '127.599998', '127.599998'),
 ('2015-06-15', '126.099998', '126.919998', '